In [26]:
import nbimporter
import dataDownloader as dd

In [32]:
def getPreviousMatches(matches,x,teamVenue,predictionMatches):
    previousMatches = []
    for iHome in range(x-1,0,-1):
        if(len(previousMatches) == predictionMatches):
            break
        if(matches[x][teamVenue]==matches[iHome]['HomeTeam'] or matches[x][teamVenue]==matches[iHome]['AwayTeam']):
            previousMatches.append(matches[iHome])
    return previousMatches

In [40]:
import glob
import csv
import random
def generateTestAndTrainData(trainToOverallRatio, predictionMatches):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    #Read files
    filesPaths = glob.glob("../data/*.csv")
    for dataFile in filesPaths: 
        with open(dataFile) as csvfile:
            reader = csv.DictReader(csvfile)
            #print(f"reading file {dataFile} with {sum(1 for line in reader)} entries")
            matches = []
            for row in reader:
                matches.append(row)
            for x in range(len(matches)):
                homePreviousMatches = getPreviousMatches(matches, x, 'HomeTeam', predictionMatches)
                awayPreviousMatches = getPreviousMatches(matches, x,'AwayTeam',predictionMatches)
                print(f"date = {matches[x]['Date']}, homePreviousMatches = {len(homePreviousMatches)}")

    print(filesPaths)

In [41]:
generateTestAndTrainData(0.7,10)

, homePreviousMatches = 10
date = 09/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 14/12/2019, homePreviousMatches = 10
date = 15/12/2019, homePreviousMatches = 10
date = 15/12/2019, homePreviousMatches = 10
date = 15/12/2019, homePreviousMatches = 10
date = 16/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 21/12/2019, homePreviousMatches = 10
date = 22/12/2019, homePreviousMatches = 10
date = 22/12/2019, homePreviousMatches = 10
date = 26/12/2019, homePreviousMatches = 10
date = 26/12/2019, homePreviousMatches = 10
date 